<a href="https://colab.research.google.com/github/lavishabhambri/Project---Facial-Expressions-Recognition/blob/master/Project_Facial_Expressions_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/muxspace/facial_expressions.git  #this clones the data for us.

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 13.82 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [3]:
import csv
data = {}  #this contains the keys - expression and the values are the images corresponding to the expression.
with open('/content/facial_expressions/data/legend.csv') as f:  #opening the legend.csv file
  reader = csv.reader(f)  #this reads the file from top, but we dont want the headers in our data, so move this reader object to the next
  next(reader)
  for row in reader:
    key = row[2].lower()  #keys - expression which are = row[0] in lower case so that 'anger' and 'ANGER' are considered same.
    if key in data:
      data[key].append(row[1])  #if it already exists in the dictionary then just append the image name as valuse
    else:
      data[key] = [row[1]]  #else make a new list with the image name.

In [4]:
emotions_list = list(data.keys())  #this gives expressions only in lower format as we did above in lowercase.
emotions_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
#Adding sub directories for traning and testing data inside master_data directory
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [7]:
#Inside the training and testing folders create the folders for expressions
for emotion in emotions_list:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

In [8]:
#Adding images to these emotions in training and testing folders
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size * len(images))
  train_images = images[:train_size]
  test_images = images[train_size: ]

  for image in train_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/training', emotion, image)
    copyfile(source, dest)

  for image in test_images:
    source = os.path.join('/content/facial_expressions/images', image)
    dest = os.path.join('/content/master_data/testing', emotion, image)
    copyfile(source, dest)

In [10]:
#Importing all the dependencies
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [12]:
#Fitting the model
model = tf.keras.models.Sequential([
                                    Conv2D(16, (3, 3), activation = 'relu', input_shape = (100, 100, 3)),
                                    MaxPooling2D(2, 2),
                                    Conv2D(32, (3, 3), activation = 'relu'),
                                    MaxPooling2D(2, 2),
                                    Conv2D(64, (3, 3), activation = 'relu'),
                                    MaxPooling2D(2, 2),
                                    Flatten(),
                                    Dense(512, activation = 'relu'),
                                    Dense(8, activation = 'softmax')
])

In [13]:
model.compile(optimizer= Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0

In [14]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

In [15]:
train_datagen = ImageDataGenerator(rescale= 1.0/255)  #normalising the data to same pixels
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128)

test_datagen = ImageDataGenerator(rescale= 1.0/255)  #normalising the data to same pixels
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [16]:
#Early Stopping callback - to stop early when the validation accuracy does not improve much further
es = EarlyStopping(monitor = 'val_acc', patience = 2, min_delta = 0.01)

In [17]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
86/86 [==============================] - 19s 222ms/step - loss: 1.2914 - accuracy: 0.5288 - val_loss: 1.0286 - val_accuracy: 0.5635
Epoch 2/10
86/86 [==============================] - 19s 216ms/step - loss: 0.8996 - accuracy: 0.6561 - val_loss: 0.9686 - val_accuracy: 0.6550
Epoch 3/10
86/86 [==============================] - 18s 213ms/step - loss: 0.7756 - accuracy: 0.7311 - val_loss: 0.8663 - val_accuracy: 0.6648
Epoch 4/10
86/86 [==============================] - 18s 214ms/step - loss: 0.7130 - accuracy: 0.7543 - val_loss: 1.1249 - val_accuracy: 0.5912
Epoch 5/10
86/86 [==============================] - 18s 211ms/step - loss: 0.6915 - accuracy: 0.7596 - val_loss: 1.0802 - val_accuracy: 0.6229
Epoch 6/10
86/86 [==============================] - 18s 213ms/step - loss: 0.6627 - accuracy: 0.7728 - val_loss: 1.2074 - val_accuracy: 0.6255
Epoch 7/10
86/86 [==============================] - 18s 212ms/step